In [10]:
import pandas as pd
import zipfile

# Name of the uploaded zip file
zip_file_name = 'ml-100k.zip'

# Extract the required files from the zip archive
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extract('ml-100k/u.data')
    zip_ref.extract('ml-100k/u.item')

# Paths to the extracted local files
path_ratings = 'ml-100k/u.data'
path_movies = 'ml-100k/u.item'

# Spaltennamen definieren
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure',
          'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
          'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

# Daten aus den lokalen Dateien einlesen
ratings = pd.read_csv(path_ratings, sep='\t', names=r_cols, encoding='latin-1')
movies = pd.read_csv(path_movies, sep='|', names=m_cols, encoding='latin-1')

# Nur die benötigten Spalten aus dem "movies" DataFrame auswählen
movie_titles = movies[['movie_id', 'title']]

# Die beiden DataFrames zusammenführen
df = pd.merge(ratings, movie_titles, on='movie_id')

print("Daten erfolgreich aus der ZIP-Datei geladen und zusammengeführt:")
print(df.head())

# Erstelle die User-Item-Matrix
movie_matrix = df.pivot_table(index='user_id', columns='title', values='rating')

# Zeige die ersten paar Zeilen und Spalten der neuen Matrix an
print("User-Item-Matrix erfolgreich erstellt:")
print(movie_matrix.head())

# Wähle einen Film aus, für den wir Empfehlungen finden wollen
movie_title = 'Star Wars (1977)'

# Hole alle Bewertungen für diesen spezifischen Film
starwars_ratings = movie_matrix[movie_title]

# Berechne die Korrelation zwischen diesem Film und allen anderen Filmen
similar_to_starwars = movie_matrix.corrwith(starwars_ratings)

# Erstelle einen DataFrame aus dem Ergebnis und entferne NaN-Werte
recommendations = pd.DataFrame(similar_to_starwars, columns=['Correlation'])
recommendations.dropna(inplace=True)


# --- Bereinigung und Verbesserung der Empfehlungen ---

# Wir brauchen eine Zählung, wie oft jeder Film bewertet wurde
ratings_count = pd.DataFrame(df.groupby('title')['rating'].count())
ratings_count.rename(columns={'rating': 'num_of_ratings'}, inplace=True)

# Füge die Bewertungsanzahl zu unserer Empfehlungs-Tabelle hinzu
recommendations = recommendations.join(ratings_count)

# Filtere Filme heraus, die weniger als 100 Bewertungen haben.
# Dies entfernt statistisch unbedeutende Ähnlichkeiten mit unbeliebten Filmen.
final_recommendations = recommendations[recommendations['num_of_ratings'] > 100].sort_values('Correlation', ascending=False)


# --- Ergebnis anzeigen ---
print(f"Empfehlungen basierend auf '{movie_title}':")
print(final_recommendations.head(10))

Daten erfolgreich aus der ZIP-Datei geladen und zusammengeführt:
   user_id  movie_id  rating  timestamp                       title
0      196       242       3  881250949                Kolya (1996)
1      186       302       3  891717742    L.A. Confidential (1997)
2       22       377       1  878887116         Heavyweights (1994)
3      244        51       2  880606923  Legends of the Fall (1994)
4      166       346       1  886397596         Jackie Brown (1997)
User-Item-Matrix erfolgreich erstellt:
title    'Til There Was You (1997)  1-900 (1994)  101 Dalmatians (1996)  \
user_id                                                                   
1                              NaN           NaN                    2.0   
2                              NaN           NaN                    NaN   
3                              NaN           NaN                    NaN   
4                              NaN           NaN                    NaN   
5                              NaN    

/usr/local/lib/python3.12/dist-packages/numpy/lib/_function_base_impl.py:2922: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.12/dist-packages/numpy/lib/_function_base_impl.py:2923: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/usr/local/lib/python3.12/dist-packages/numpy/lib/_function_base_impl.py:2914: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.12/dist-packages/numpy/lib/_function_base_impl.py:2773: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.12/dist-packages/numpy/lib/_function_base_impl.py:2773: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


Empfehlungen basierend auf 'Star Wars (1977)':
                                                    Correlation  \
title                                                             
Star Wars (1977)                                       1.000000   
Empire Strikes Back, The (1980)                        0.747981   
Return of the Jedi (1983)                              0.672556   
Raiders of the Lost Ark (1981)                         0.536117   
Austin Powers: International Man of Mystery (1997)     0.377433   
Sting, The (1973)                                      0.367538   
Indiana Jones and the Last Crusade (1989)              0.350107   
Pinocchio (1940)                                       0.347868   
Frighteners, The (1996)                                0.332729   
L.A. Confidential (1997)                               0.319065   

                                                    num_of_ratings  
title                                                               
Star Wars 